# Save per insurance company

In [7]:
from azure.cosmos import CosmosClient
from cryptography.fernet import Fernet
import json
import os

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

with open("fernet.key", "rb") as f:
    fernet_key = f.read()
cipher = Fernet(fernet_key)

client = CosmosClient(url, credential=key)

company_to_container = {
    "BH": "bh_collection",
    "CNAM": "cnam_collection",
    "STAR": "star_collection"
}

insurance_company = "BH"  

if insurance_company not in company_to_container:
    raise ValueError(f"Unknown insurance company: {insurance_company}")

database = client.get_database_client("cosmicworks")
container_name = company_to_container[insurance_company]
container = database.get_container_client(container_name)

json_file_path = f"cropped_fields/extracted_text.json"
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"JSON file for {insurance_company} not found.")

with open(json_file_path, "r") as file:
    data = json.load(file)

encrypted = cipher.encrypt(json.dumps(data).encode()).decode()

container.create_item(
    body={"category": "text_data", "insurance": insurance_company, "encrypted_data": encrypted},
    enable_automatic_id_generation=True
)

print(f"Encrypted data for {insurance_company} saved to {container_name}.")


Encrypted data for BH saved to bh_collection.


In [9]:
from azure.cosmos import CosmosClient
from cryptography.fernet import Fernet, InvalidToken
import json

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

# Load Fernet key
with open("fernet.key", "rb") as f:
    fernet_key = f.read()

cipher = Fernet(fernet_key)

client = CosmosClient(url, credential=key)
database = client.get_database_client("cosmicworks")
CONTAINER_NAMES = ["bh_collection", "star_collection", "cnam_collection"]

for container_name in CONTAINER_NAMES:
    print(f"\nFetching from {container_name}...")
    container = database.get_container_client(container_name)
    
    query = "SELECT * FROM c"
    items = container.query_items(query=query, enable_cross_partition_query=True)
    
    for item in items:
        encrypted = item.get("encrypted_data")
        if encrypted:
            try:
                decrypted_data = cipher.decrypt(encrypted.encode()).decode()
                parsed = json.loads(decrypted_data)
                print(json.dumps(parsed, indent=4))
            except InvalidToken:
                print(f"Could not decrypt item with id: {item.get('id')}")
        else:
            print(f"No 'encrypted_data' field in item with id: {item.get('id')}")



Fetching from bh_collection...
{
    "cin_passport_subscriber": "98765432",
    "patient_name": "Mohamed Mouhamed",
    "subscriber_name": "Abderazak Braham",
    "subscriber_address": "Sfax"
}

Fetching from star_collection...
{
    "cin_passport_subscriber": "98765432",
    "patient_name": "Mohamed Mouhamed",
    "subscriber_name": "Abderazak Braham",
    "subscriber_address": "Sfax"
}

Fetching from cnam_collection...
{
    "cin_passport_subscriber": "98765432",
    "patient_name": "Mohamed Mouhamed",
    "subscriber_name": "Abderazak Braham",
    "subscriber_address": "Sfax"
}
